#  WEBSCRAPPING autoscout24

- Importe de librerías necesarias

In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.common.by import By

from tqdm import tqdm

import re
import time

import pandas as pd

- Creamos la configuración del navegador

In [2]:
c_options = webdriver.ChromeOptions()
c_options.add_argument("--start-maximized")
driver = webdriver.Chrome(options=c_options)

- Establecemos las configuraciones de la URL para buscar con los diferentes filtros

In [3]:
base_url = "https://www.autoscout24.es/lst/tesla/"
tesla_models = ["model-3", "model-s", "model-x", "model-y"]

fixed_params = (
    "atype=C&damaged_listing=exclude&desc=0&powertype=kw"
    "&search_id=a1ps9r8ye6&sort=standard&source=homepage_search-mask&ustate=N%2CU"
)

bcol_values = ['bc_azul', 'bc_rojo', 'bc_blanco', 'bc_negro', 'bc_gris']
cy_values = ["E", "F", "I"]

country_mapping = {
    "E": "España",
    "F": "Francia",
    "I": "Italia"
}

results = []

accept_cookies = False

patron_tesla = re.compile(r'\bTesla\b', re.IGNORECASE)
patron_modelo = re.compile(r'\bModel (3|S|X|Y)\b', re.IGNORECASE)


- Realizamos el Scrapeo de las diferentes páginas

In [ ]:
try:
    for model in tqdm(tesla_models):
        for bcol in bcol_values:
            for cy in cy_values:
                query_params = f"cy={cy}"
                full_url = f"{base_url}{model}/{bcol}?{fixed_params}&{query_params}"
                driver.get(full_url)

                if not accept_cookies:
                    try:
                        wait = WebDriverWait(driver, 10)
                        cookie_button = wait.until(EC.element_to_be_clickable((By.CLASS_NAME, "_consent-accept_1lphq_114")))
                        cookie_button.click()
                        print("Cookies aceptadas correctamente.")
                        accept_cookies = True
                    except TimeoutException:
                        print("No se encontraron cookies para aceptar.")

                while True:
                    time.sleep(3)
                    car_cards = driver.find_elements(By.TAG_NAME, 'article')
                    if len(car_cards) > 0:
                        for car in car_cards:
                            try:
                                marca_texto = car.find_element(By.TAG_NAME, 'h2').text
                                marca = patron_tesla.search(marca_texto)
                                modelo = patron_modelo.search(marca_texto)
                            except NoSuchElementException:
                                marca_texto = "No disponible"
                                marca = None
                                modelo = None

                            try:
                                descripcion = car.find_element(By.CLASS_NAME, 'ListItem_subtitle__VEw08').text
                            except NoSuchElementException:
                                descripcion = "No disponible"

                            try:
                                precio = car.find_element(By.CSS_SELECTOR, '[data-testid="regular-price"]').text
                                precio_limpio = re.sub(r'[^\d.]', '', precio)
                                precio = float(precio_limpio.replace('.', ''))
                            except NoSuchElementException:
                                precio = "No disponible"

                            try:
                                kilometraje = car.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-mileage_road"]').text
                                kilometraje_limpio = re.sub(r'[^\d.]', '', kilometraje)
                                kilometraje = int(kilometraje_limpio.replace('.', ''))
                            except NoSuchElementException:
                                kilometraje = "No disponible"

                            try:
                                trasmision = car.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-transmission"]').text
                            except NoSuchElementException:
                                trasmision = "No disponible"

                            try:
                                combustible = car.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-gas_pump"]').text
                            except NoSuchElementException:
                                combustible = "No disponible"

                            try:
                                velocidad = car.find_element(By.CSS_SELECTOR, '[data-testid="VehicleDetails-speedometer"]').text
                            except NoSuchElementException:
                                velocidad = "No disponible"

                            try:
                                vendedor = car.find_element(By.CLASS_NAME, 'SellerInfo_private__THzvQ').text
                            except NoSuchElementException:
                                vendedor = "No disponible"

                            pais_nombre = country_mapping.get(cy, "Desconocido")

                            # Extraer texto de las coincidencias o manejar valores no encontrados
                            marca_final = marca.group(0) if marca else "No disponible"
                            modelo_final = modelo.group(0) if modelo else "No disponible"

                            results.append({
                                "marca": marca_final,
                                "modelo": modelo_final,
                                "descripcion": descripcion,
                                "precio": precio,
                                "color": bcol.split('_')[1].capitalize(),
                                "pais": cy,
                                "kilometraje": kilometraje,
                                "trasmision": trasmision,
                                "combustible": combustible,
                                "velocidad": velocidad,
                                "vendedor": vendedor
                            })

                        try:
                            next_button = driver.find_element(By.CLASS_NAME, "prev-next")
                            if "pagination-item--disabled" in next_button.get_attribute("class"):
                                print("Botón 'Siguiente' deshabilitado. Fin del paginador.")
                                break
                            else:
                                next_button.click()
                                print("Pasando a la siguiente página del paginador.")
                        except NoSuchElementException:
                            print("No hay más páginas en el paginador.")
                            break


finally:
    driver.quit()


In [ ]:
df_tesla = pd.DataFrame(results)
# df_tesla.to_csv("tesla_autoscout24.csv", index=True, encoding="utf-8-sig")

df_tesla.head(5)